In [1]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=0)

## Day 1

In part 2, increment of sum of three consecutive numbers is determined from the sliding window of **four** elements. The head of the four elements is going out, and the tail element is incomming.  

In [14]:
# day 1, part 1

import itertools
# python 3.10 introduces itertools.pairwise

with open("./2021/day01_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    count = sum(1 for a, b in itertools.pairwise(xs) if a < b)
    print(count)

1752


In [15]:
# day 1, part 2

import itertools
import collections
from typing import Iterable, Generator, Any

# https://docs.python.org/3/library/itertools.html#itertools-recipes
def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) -> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(itertools.islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)

with open("./2021/day01_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    count = sum(1 for head, _, _, tail in sliding_window(xs, 4) if head < tail)
    print(count)

1781


## Day 2

Discuss change in submarine's position both horizontally and vertically (as depth). The part 2 introduces "aim" as an additional variable.

In [3]:
from typing import Iterable

def day02_part1(xs: Iterable[tuple[str, int]]) -> int:
    """
    """
    h, d = 0, 0
    for cmd, amount in xs:
        if cmd == "forward":
            h += amount
        elif cmd == "down":
            d += amount
        elif cmd == "up":
            d -= amount
        else:
            raise ValueError(f"Unknown command: {cmd} {amount}")
    return h * d

In [5]:
test = """
forward 5
down 5
forward 8
up 3
down 8
forward 2
"""
xs = [s.split() for s in test.split("\n") if s.strip()]
xs = [(cmd, int(w)) for cmd, w in xs]
day02_part1(xs)

150

In [6]:
with open("./2021/day02_input.txt") as f:
    xs = [s.split() for s in f.readlines() if s.strip()]
    xs = [(cmd, int(w)) for cmd, w in xs]
    res = day02_part1(xs)
    print(res)

1804520


In [7]:
def day02_part2(xs: Iterable[tuple[str, int]]) -> int:
    h = 0
    d = 0
    aim = 0
    for cmd, amount in xs:
        if cmd == "forward":
            h += amount
            d += amount * aim
        elif cmd == "down":
            aim += amount
        elif cmd == "up":
            aim -= amount
        else:
            raise ValueError(f"Unknown command: {cmd} {amount}")
    return h * d

In [8]:
test = """
forward 5
down 5
forward 8
up 3
down 8
forward 2
"""
xs = [s.split() for s in test.split("\n") if s.strip()]
xs = [(cmd, int(w)) for cmd, w in xs]
day02_part2(xs)

900

In [9]:
with open("./2021/day02_input.txt") as f:
    xs = [s.split() for s in f.readlines() if s.strip()]
    xs = [(cmd, int(w)) for cmd, w in xs]
    res = day02_part2(xs)
    print(res)


1971095320


## Day 3: Binary manipulations

In [41]:
import collections
from typing import Iterable


def day03_part1(content: str) -> int:

    def most_common(xs: Iterable[str]) -> str:
        """Assert xs contains either 0 or 1"""
        cnt = collections.Counter(xs)
        return str(int(cnt['0'] <= cnt['1']))
    
    lines = [list(line.strip()) for line in content.split() if line.strip()]
    gamma = "".join(most_common(seq) for seq in zip(*lines))
    gamma = int(gamma, 2)
    bin_length = len(lines[0])
    epsilon = ((1 << bin_length) - 1) ^ gamma
    print(f"{(gamma, epsilon) = }")
    return gamma * epsilon


In [42]:
s = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
"""

day03_part1(s)

(gamma, epsilon) = (22, 9)


198

In [43]:
with open("./2021/day03_input.txt") as f:
    res = day03_part1(f.read())
    print(res)

(gamma, epsilon) = (190, 3905)
741950


In [44]:
import numpy as np
from typing import Iterable
import collections

def day03_part2(content: str) -> int:

    def most_common(xs: Iterable[int]) -> int:
        """Assert xs contains either 0 or 1"""
        cnt = collections.Counter(xs)
        return int(cnt[0] <= cnt[1])
    

    def _rating(zs, n_digits, cmp) -> int:
        xs = np.copy(zs)
        
        i = n_digits - 1
        while len(xs) > 1:
            digits = (xs >> i) & 1
            criteria = cmp(digits, most_common(digits))
            xs = xs[criteria]
            i -= 1
        return xs[0]

    def oxygen_rating(zs, n_digits) -> int:
        return _rating(zs, n_digits, np.equal)

    def co2_rating(zs, n_digits) -> int:
        return _rating(zs, n_digits, np.not_equal)
    
    lines = [line.strip() for line in content.split() if line.strip()]
    xs = np.array([int(line, 2) for line in lines])

    n_digits = len(lines[0])
    oxygen = oxygen_rating(xs, n_digits)
    co2 = co2_rating(xs, n_digits)
    print(f"{(oxygen, co2) = }")
    return oxygen * co2


In [45]:
s = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
"""

day03_part2(s)

(oxygen, co2) = (23, 10)


230

In [46]:
with open("./2021/day03_input.txt") as f:
    res = day03_part2(f.read())
    print(res)

(oxygen, co2) = (282, 3205)
903810


## Day 4: 5 x 5 Bingo

**NOTE**: diagonals don't count!

In [133]:
from typing import Generator, Any, Iterable, Optional
import collections


def day04_parse(content: str) -> tuple[list[int], list[list[list[int]]]]:
    """Return integer sequence AND a list of 5x5 bingos"""
    chunks = [chunk.strip() for chunk in content.split("\n\n") if chunk.strip()]
    xs = [int(s) for s in chunks[0].split(",") if s.strip()]
    dominos = [
        [
            [int(s) for s in line.split()]
            for line in chunk.split("\n") if line.strip()
        ] 
        for chunk in chunks[1:]
    ]
    return xs, dominos


def day04_candidates(domino: list[list[Any]]) -> Generator[frozenset[Any], None, None]:
    """
    """
    for xs in domino:
        yield frozenset(xs)
    
    for xs in zip(*domino):
        yield frozenset(xs)

def day04_part1(calls: Iterable[int], dominos: list[list[list[int]]]) -> Optional[int]:

    d = collections.defaultdict(list)
    for i, domino in enumerate(dominos):
        for candidate in day04_candidates(domino):
            d[candidate].append(i)
        
    called = set()
    last_x = -1   # placeholder for scope purpose
    for x in calls:
        called.add(x)
        if any(candidate <= called for candidate in d):
            last_x = x
            break

    for candidate in d:
        if candidate <= called:
            idx = d[candidate][0]  # assume the winner is unique
            board = frozenset({x for xs in dominos[idx] for x in xs})
            print(f"{idx = },  {last_x = },  {candidate = },  {called = }")
            return sum(board - called) * last_x
        
    print("something is wrong!")
    return None

In [134]:
s = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

calls, dominos = day04_parse(s)
day04_part1(calls, dominos)

idx = 2,  last_x = 24,  candidate = frozenset({4, 14, 17, 21, 24}),  called = {0, 2, 4, 5, 7, 9, 11, 14, 17, 21, 23, 24}


4512

In [135]:
with open("./2021/day04_input.txt") as f:
    calls, dominos = day04_parse(f.read())
    res = day04_part1(calls, dominos)
    print(res)

idx = 88,  last_x = 14,  candidate = frozenset({71, 8, 14, 23, 24}),  called = {3, 8, 9, 12, 14, 18, 22, 23, 24, 30, 37, 40, 45, 53, 54, 57, 58, 62, 70, 71, 72, 73, 80, 81, 88, 95, 98}
10374


In [139]:
def day04_part2(calls: Iterable[int], dominos: list[list[list[int]]]) -> Optional[int]:

    d = collections.defaultdict(list)
    for i, domino in enumerate(dominos):
        for candidate in day04_candidates(domino):
            d[candidate].append(i)
    
    n_boards = len(dominos)
    print(f"{n_boards = }")
    won_indices = []
    called: set[int] = set()
    done = set()
    last_x = -1

    for x in calls:
        called.add(x)
        for candidate, indices in d.items():
            if candidate not in done and candidate <= called:
                print(set(candidate), indices)
                last_x = x
                for idx in indices:
                    if idx not in won_indices:
                        won_indices.append(idx)
                done.add(candidate)
    
        if len(won_indices) == n_boards:
            break

    if won_indices:
        idx = won_indices[-1]
        board = frozenset({x for xs in dominos[idx] for x in xs})
        print(f"{idx = },  {last_x = }, {called = }")
        return sum(board - called) * last_x
    
    print("something is wrong!")
    return None

In [140]:
s = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

calls, dominos = day04_parse(s)
day04_part2(calls, dominos)

n_boards = 3
{17, 4, 21, 24, 14} [2]
{16, 21, 7, 9, 14} [0]
{0, 16, 7, 10, 13} [1]
idx = 1,  last_x = 13, called = {0, 2, 4, 5, 7, 9, 10, 11, 13, 14, 16, 17, 21, 23, 24}


1924

In [141]:
with open("./2021/day04_input.txt") as f:
    calls, dominos = day04_parse(f.read())
    res = day04_part2(calls, dominos)
    print(res)

n_boards = 100
{23, 71, 8, 24, 14} [88]
{0, 34, 55, 45, 95} [31]
{34, 30, 8, 29, 14} [46]
{0, 66, 88, 12, 29} [62]
{3, 22, 9, 58, 27} [52]
{55, 88, 73, 27, 28} [48]
{53, 23, 72, 28, 45} [81]
{66, 19, 37, 87, 24} [36]
{34, 82, 54, 87, 14} [77]
{29, 83, 71, 45, 14} [9]
{83, 55, 73, 58, 9} [10]
{80, 48, 66, 83, 58} [48]
{96, 36, 23, 88, 31} [20]
{98, 36, 53, 88, 29} [35]
{18, 82, 54, 22, 26} [9]
{81, 8, 24, 26, 31} [97]
{36, 9, 63, 14, 95} [4]
{36, 71, 8, 9, 63} [21]
{66, 19, 23, 31, 63} [60]
{21, 22, 14, 73, 9} [9]
{37, 21, 71, 22, 73} [17]
{96, 19, 21, 70, 63} [86]
{82, 53, 21, 72, 9} [86]
{33, 3, 87, 73, 28} [3]
{80, 33, 81, 70, 62} [23]
{48, 33, 36, 70, 40} [61]
{33, 70, 12, 46, 95} [84]
{83, 70, 86, 87, 24} [1]
{26, 36, 86, 58, 29} [9]
{81, 66, 86, 55, 62} [48]
{32, 19, 37, 21, 45} [0]
{32, 98, 86, 9, 63} [9]
{32, 82, 71, 24, 12} [9]
{32, 66, 21, 9, 62} [17]
{32, 96, 81, 5, 53} [40]
{0, 34, 5, 56, 63} [23]
{98, 37, 56, 95, 63} [50]
{33, 98, 86, 56, 26} [56]
{48, 71, 72, 56, 29} [81]


## Day 5: いもす法

In [183]:
import collections
Coord = tuple[int, int]

def day05_parse(content: str) -> list[tuple[Coord, Coord]]:
    lines = [line.strip().split(" -> ") for line in content.split("\n") if line.strip()]
    res = []
    for from_, to_ in lines:
        x1, y1 = map(int, from_.split(","))
        x2, y2 = map(int, to_.split(","))
        res.append(((x1, y1), (x2, y2)))
    return res


def day05_part1(pairs: list[tuple[Coord, Coord]]) -> int:
    x_ub = max(max(x1, x2) for (x1, _), (x2, _) in pairs)
    y_ub = max(max(y1, y2) for (_, y1), (_, y2) in pairs)

    grid_v = collections.defaultdict(int)
    grid_h = collections.defaultdict(int)
    for (x1, y1), (x2, y2) in pairs:
        if y1 == y2:
            grid_v[min(x1, x2), y1] += 1
            grid_v[max(x1, x2) + 1, y2] -= 1
        elif x1 == x2:
            grid_h[x1, min(y1, y2)] += 1
            grid_h[x2, max(y1, y2) + 1] -= 1    

    grid = collections.defaultdict(int)
    cnt = 0
    for i in range(x_ub + 2):
        for j in range(y_ub + 2):
            grid_v[i, j] += grid_v[i - 1, j]
            grid_h[i, j] += grid_h[i, j - 1]
            grid[i, j] = grid_v[i, j] + grid_h[i, j]
            if grid[i, j] > 1:
                cnt += 1
    return cnt


def day05_display(grid):
    x_ub = max(x for (x, _) in grid.keys())
    y_ub = max(y for (_, y) in grid.keys())

    s = "\n".join(
        " ".join(str(grid[i, j]) for i in range(x_ub + 2))
        for j in range(y_ub + 2)
    )
    
    print(s)
    print()

In [184]:
content = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
"""

pairs = day05_parse(content)
day05_part1(pairs)

5

In [185]:
import math

def day05_part2(pairs: list[tuple[Coord, Coord]], show: bool=False) -> int:
    x_ub = max(max(x1, x2) for (x1, _), (x2, _) in pairs)
    y_ub = max(max(y1, y2) for (_, y1), (_, y2) in pairs)

    grid = collections.defaultdict(int)
    for (x1, y1), (x2, y2) in pairs:
        d = math.gcd(x2 - x1, y2 - y1)
        for i in range(d + 1):
            x = x1 + i * (x2 - x1) // d
            y = y1 + i * (y2 - y1) // d
            grid[x, y] += 1

    cnt = 0
    for i in range(x_ub + 2):
        for j in range(y_ub + 2):
            if grid[i, j] > 1:
                cnt += 1
    
    if show:
        day05_display(grid)
    return cnt


In [186]:
content = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
"""

pairs = day05_parse(content)
day05_part2(pairs, show=True)

1 0 1 0 0 0 0 1 1 0 0 0
0 1 1 1 0 0 0 2 0 0 0 0
0 0 2 0 1 0 1 1 1 0 0 0
0 0 0 1 0 2 0 2 0 0 0 0
0 1 1 2 3 1 3 2 1 1 0 0
0 0 0 1 0 2 0 0 0 0 0 0
0 0 1 0 0 0 1 0 0 0 0 0
0 1 0 0 0 0 0 1 0 0 0 0
1 0 0 0 0 0 0 0 1 0 0 0
2 2 2 1 1 1 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0



12

In [182]:
with open("./2021/day05_input.txt") as f:
    pairs = day05_parse(f.read())
    res = day05_part2(pairs)
    print(res)

21101


## Day 6: Lanternfish
Let n[t, r] be the number of lanternfish with remainder count r at time t. Consider change in state from (t-1), we have

```
n[t, 8] = n[t - 1, 0]
n[t, 7] = n[t - 1, 8]
n[t, 6] = n[t - 1, 7] + n[t - 1, 0]
n[t, r] = n[t - 1, r + 1]    (for 0 <= r <= 5)
```

In [35]:
import collections

def day06_f(n: dict[tuple[int, int], int], t: int):
    n[t, 8] = n[t - 1, 0]
    n[t, 7] = n[t - 1, 8]
    n[t, 6] = n[t - 1, 7] + n[t -1, 0]
    for i in range(6):
        n[t, i] = n[t - 1, i + 1]

def day06(xs: list[int], days=80) -> int:
    d = collections.defaultdict(int)

    counts = collections.Counter(xs)
    for r, count in counts.items():
        d[0, r] = count
    
    for t in range(1, days + 1):
        day06_f(d, t)
    
    # for t in range(19):
    #     print(f"{t}: {[d[t, r] for r in range(9)]}")

    res = sum(d[days, r] for r in range(9))
    return res

In [36]:
content = "3,4,3,1,2"
xs = [int(c) for c in content.split(",")]
day06(xs, 80)

5934

In [37]:
with open("./2021/day06_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day06(xs, days=80)
    print(res)

362740


In [38]:
with open("./2021/day06_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day06(xs, days=256)
    print(res)

1644874076764


## Day 7: Crab positions

Minimum location p satisfying $\sum_i |p - x_i|$ is given from the median of $x_i$ (with $i = 0, 1, 2, \dots$). 

In [44]:
from typing import Union

def day07_median(xs: list[int]) -> int:
    """Find median of xs. If the number of xs is even, pick smaller one."""
    n = len(xs)
    k = n // 2
    xs.sort()
    return xs[k]

def day07_part1(xs: list[int]) -> Union[int, float]:
    """Find median of xs"""
    med = day07_median(xs)
    res = sum(abs(med - x) for x in xs)
    return res

In [45]:
xs = [16,1,2,0,4,2,7,1,2,14]
day07_part1(xs)

37

In [76]:
with open("./2021/day07_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day07_part1(xs)
    print(res)

344735


In Part 2, the cost $f(n)$ of $n$ steps is $f(n) = 1 + 2 + ... + n = \frac{n (n + 1)}{2}$.

Find position $p$ minimizing $\sum_i f(\lvert p - x_i \rvert)$, which is to find $p$ minimizing  $\sum_i \left[ (p - x_i)^2 + \lvert p - x_i \rvert \right] $. Hence

$$
\frac{d f}{d p} = 2 \sum_i (p - x_i) + \sum_i \mathrm{sgn}(p - x_i) = 0
$$

Assuming the funciton $\frac{d f}{d p} = 2 N p - \sum_i x_i + \sum_i \mathrm{sgn} (p - x_i)$ is monotonically increasing, find $p$ satisfying $g(p) = 0$ with binary search. 

Note that the solution of the binary search $p_\mathrm{sol}$ is an integer satisfying $g(p_\mathrm{sol}) \geq 0$. It is possible that $p = (p_\mathrm{sol} - 1)$ giving the minimum $f(p)$ if the inequality holds.

In [125]:
import bisect

def binary_search(xs):
    def is_ok(index):
        return day07_dfdp(index, xs) >= 0

    ng = -1
    ok = 1000000

    while abs(ok - ng) > 1:
        mid = (ok + ng) // 2
        if is_ok(mid):
            ok = mid
        else:
            ng = mid
    return ok

def day07_sgn_sum(p, xs):
    """Return sum_i sgn(p - x_i). Assume xs is sorted."""
    n = len(xs)
    idx_left = bisect.bisect_left(xs, p)
    idx_right = bisect.bisect_right(xs, p)
    # print(f"{idx_left = }")
    # print(f"{idx_right = }")
    return  idx_left + idx_right - n

def day07_dfdp(p: int, xs: list[int]):
    np = len(xs) * p
    sum_xs = sum(xs)
    sum_sgn = day07_sgn_sum(p, xs)
    return 2 * (np - sum_xs) + sum_sgn


def day07_cost(p: int, xs: list[int]) -> int:
    res = 0
    for x in xs:
        d = abs(p - x)
        res += d * (d + 1) // 2
    return res    

def day07_part2(xs: list[int]):
    xs.sort()
    p = binary_search(xs)
    res = day07_cost(p, xs)
    if (tmp := day07_cost(p - 1, xs)) < res:
        res = tmp
        p = p - 1
    print(f"{p = }")

    return res

In [126]:
xs = [16,1,2,0,4,2,7,1,2,14]
day07_part2(xs)

p = 5


168

In [127]:
with open("./2021/day07_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day07_part2(xs)
    print(res)

p = 474
96798233
